In [9]:
#current working code with NA filtered out. 

In [ ]:
import xlwings as xw
import requests
import pandas as pd

wb = xw.Book('Richspread.xlsx')       
ticker_sht    = wb.sheets['Ticker']
annual_sht    = wb.sheets['Annual']
quarterly_sht = wb.sheets['Quarterly']

symbol  = ticker_sht.range('B2').value    
api_key = ''              

ov_url      = (
    f'https://www.alphavantage.co/query'
    f'?function=OVERVIEW'
    f'&symbol={symbol}'
    f'&apikey={api_key}'
)
overview     = requests.get(ov_url).json()
description  = overview.get('Description', 'No description available')

ticker_sht.range('A3').value = 'Description'
ticker_sht.range('A4').value = description

def fetch_df(function_name: str, report_type: str) -> pd.DataFrame:
    url = (
        f'https://www.alphavantage.co/query'
        f'?function={function_name}'
        f'&symbol={symbol}'
        f'&apikey={api_key}'
    )
    payload = requests.get(url).json()
    if 'Note' in payload or 'Error Message' in payload:
        print(f"⚠️ {function_name} {report_type}:",
              payload.get('Note') or payload.get('Error Message'))
        return pd.DataFrame()
    return pd.json_normalize(payload.get(report_type, []))

inc_ann = fetch_df('INCOME_STATEMENT', 'annualReports')
inc_qtr = fetch_df('INCOME_STATEMENT', 'quarterlyReports')
bs_ann  = fetch_df('BALANCE_SHEET',    'annualReports')
bs_qtr  = fetch_df('BALANCE_SHEET',    'quarterlyReports')
cf_ann  = fetch_df('CASH_FLOW',        'annualReports')
cf_qtr  = fetch_df('CASH_FLOW',        'quarterlyReports')

def clean_df(df):
    df.replace(to_replace=r'(?i)^none$', value=0, regex=True, inplace=True)
    df.fillna(0, inplace=True)
    return df

# =============================
# ANNUAL SHEET
# =============================
# Clear old data (adjust columns as needed)
annual_sht.range('A2:AZ44').clear_contents()   # Income Statement – Annual
annual_sht.range('A45:AZ89').clear_contents()  # Balance Sheet – Annual
annual_sht.range('A90:AZ150').clear_contents() # Cash Flow – Annual

# Annual Income Statement at A2
annual_sht.range('A2').value = 'Income Statement – Annual'
if not inc_ann.empty:
    inc_ann = clean_df(inc_ann)
    annual_sht.range('A3').options(index=False).value = inc_ann
else:
    annual_sht.range('A3').value = 'No data returned'

# Annual Balance Sheet at A45
annual_sht.range('A45').value = 'Balance Sheet – Annual'
if not bs_ann.empty:
    bs_ann = clean_df(bs_ann)
    annual_sht.range('A46').options(index=False).value = bs_ann
else:
    annual_sht.range('A46').value = 'No data returned'

# Annual Cash Flow at A90
annual_sht.range('A90').value = 'Cash Flow – Annual'
if not cf_ann.empty:
    cf_ann = clean_df(cf_ann)
    annual_sht.range('A91').options(index=False).value = cf_ann
else:
    annual_sht.range('A91').value = 'No data returned'

# =============================
# QUARTERLY SHEET
# =============================
quarterly_sht.range('A2:AZ99').clear_contents()    # Income Statement – Quarterly
quarterly_sht.range('A100:AZ199').clear_contents() # Balance Sheet – Quarterly
quarterly_sht.range('A200:AZ260').clear_contents() # Cash Flow – Quarterly

# Quarterly Income Statement at A2
quarterly_sht.range('A2').value = 'Income Statement – Quarterly'
if not inc_qtr.empty:
    inc_qtr = clean_df(inc_qtr)
    quarterly_sht.range('A3').options(index=False).value = inc_qtr
else:
    quarterly_sht.range('A3').value = 'No data returned'

# Quarterly Balance Sheet at A100
quarterly_sht.range('A100').value = 'Balance Sheet – Quarterly'
if not bs_qtr.empty:
    bs_qtr = clean_df(bs_qtr)
    quarterly_sht.range('A101').options(index=False).value = bs_qtr
else:
    quarterly_sht.range('A101').value = 'No data returned'

# Quarterly Cash Flow at A200
quarterly_sht.range('A200').value = 'Cash Flow – Quarterly'
if not cf_qtr.empty:
    cf_qtr = clean_df(cf_qtr)
    quarterly_sht.range('A201').options(index=False).value = cf_qtr
else:
    quarterly_sht.range('A201').value = 'No data returned'

/var/folders/rx/whpr72693f3cgk151m8h7fh80000gn/T/ipykernel_71156/997900003.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace=r'(?i)^none$', value=0, regex=True, inplace=True)
